In [14]:
import pandas as pd
import io
import folium
from folium.plugins import HeatMap
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import sys


In [2]:
def parse_price(price_str):
    """
    Átalakítja az ár stringet (pl. "62,03 M Ft", "1.06 Mrd Ft")
    numerikus értékké millió Forintban.
    """
    if pd.isna(price_str):
        return np.nan

    try:
        price_str = str(price_str).strip()

        value = np.nan

        if 'Mrd Ft' in price_str:
            num_str = price_str.replace(' Mrd Ft', '').replace(',', '.')
            value = float(num_str) * 1000
        elif 'M Ft' in price_str:
            num_str = price_str.replace(' M Ft', '').replace(',', '.')
            value = float(num_str)
        else:
             num_str = price_str.replace(',', '.')
             value = float(num_str)

        return value

    except ValueError:
        return np.nan

In [3]:
try:
    csv_filename = "data_geocoded_partial.csv"
    df = pd.read_csv(csv_filename)

    price_column_name = 'price'
    required_columns = ['latitude', 'longitude', price_column_name]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"A CSV fájlnak tartalmaznia kell a következő oszlopokat: {required_columns}")

    df['price_millions'] = df[price_column_name].apply(parse_price)

except FileNotFoundError:
    print(f"Hiba: A megadott CSV fájl ({csv_filename}) nem található.")
    exit()
except ValueError as ve:
    print(f"Hiba az adatok feldolgozása során: {ve}")
    exit()
except Exception as e:
    print(f"Váratlan hiba történt a CSV fájl beolvasása vagy az alapvető ellenőrzés közben: {e}")
    exit()

Adatbeolvasás és tisztítás:

In [5]:
map_center = [47.4979, 19.0402]
df_cleaned_base = df.dropna(subset=['latitude', 'longitude', 'price_millions']).copy()


In [12]:
print("\n--- Térkép 1: Ár/Négyzetméter Hőtérkép ---")
df_viz1 = df_cleaned_base.dropna(subset=['sqm']).copy()
df_viz1 = df_viz1[df_viz1['sqm'] > 0]
if not df_viz1.empty:
    df_viz1['price_per_sqm_mFt'] = (df_viz1['price_millions'] / df_viz1['sqm']) # Millió Ft / m²
    max_val_cap = df_viz1['price_per_sqm_mFt'].quantile(0.95)
    df_viz1 = df_viz1.dropna(subset=['price_per_sqm_mFt'])
    df_viz1.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_viz1 = df_viz1.dropna(subset=['price_per_sqm_mFt'])

    if not df_viz1.empty:
        map_price_per_sqm = folium.Map(location=map_center, zoom_start=12)
        heat_data_sqm = df_viz1[['latitude', 'longitude', 'price_per_sqm_mFt']].values.tolist()

        HeatMap(heat_data_sqm,
                name='Ár/m² (Millió Ft/m²)',
                radius=10,
                blur=8,
                min_opacity=0.3,
               ).add_to(map_price_per_sqm)

        folium.LayerControl().add_to(map_price_per_sqm)
        print("Finomhangolt Ár/nm hőtérkép elkészült.")

    else:
        print("Hiba: Nincs érvényes adat az ár/nm térképhez a számítás után.")
else:
    print("Hiba: Nincs érvényes adat (sqm) az ár/nm térképhez.")


--- Térkép 1: Ár/Négyzetméter Hőtérkép ---
Finomhangolt Ár/nm hőtérkép elkészült.


In [4]:
df_cleaned = df.dropna(subset=['latitude', 'longitude', 'price_millions']).copy()

if df_cleaned.empty:
    exit()

budapest_map = folium.Map(location=map_center, zoom_start=12)

heat_data = df_cleaned[['latitude', 'longitude', 'price_millions']].values.tolist()

HeatMap(heat_data,
        name='Ingatlan Árak (Hőtérkép)',
        radius=10,
        blur=1
       ).add_to(budapest_map)

folium.LayerControl().add_to(budapest_map)

try:
    map_filename = "budapest_ingatlan_heatmap.html"
    budapest_map.save(map_filename)
    print(f"Térkép sikeresen mentve a '{map_filename}' fájlba.")
except Exception as e:
    print(f"Hiba történt a térkép mentése közben: {e}")


Térkép sikeresen mentve a 'budapest_ingatlan_heatmap.html' fájlba.


In [24]:
# --- Ár/Négyzetméter Színezett Kör Jelölőkkel (Piros Skála) ---

if 'df_viz1' in locals() and not df_viz1.empty and 'matplotlib' in sys.modules:
    map_price_per_sqm_circles_red = folium.Map(location=map_center, zoom_start=12)
    folium.TileLayer(
        tiles='CartoDB positron',
        opacity=0.85,
        name='Halvány alapréteg'
    ).add_to(map_price_per_sqm_circles_red)


    min_val = df_viz1['price_per_sqm_mFt'].quantile(0.01)
    max_val = df_viz1['price_per_sqm_mFt'].quantile(0.99)
    norm = colors.Normalize(vmin=min_val, vmax=max_val, clip=True)

    cmap = cm.get_cmap('Reds')

    def get_color_hex(value, cmap, norm):
        rgba = cmap(norm(value))
        return colors.rgb2hex(rgba)

    for _, row in df_viz1.iterrows():
        price_sqm = row['price_per_sqm_mFt']
        marker_color = get_color_hex(price_sqm, cmap, norm)

        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=3,
            color=None,
            weight=0,
            fill=True,
            fill_color=marker_color,
            fill_opacity=0.5,
            tooltip=f"Ár/m²: {price_sqm:.2f} M Ft/m²"
        ).add_to(map_price_per_sqm_circles_red)

    legend_html = '''
    <div style="position: fixed;
                bottom: 50px; left: 50px; width: 150px; height: 130px;
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color:white; opacity: 0.85;
                ">
    &nbsp;<b>Ár/nm (M Ft/m²)</b><br>
    '''
    legend_steps = 5
    for i in range(legend_steps):
        val = min_val + (max_val - min_val) * i / (legend_steps - 1)
        color = get_color_hex(val, cmap, norm)
        legend_html += f'&nbsp;<i class="fa fa-square" style="color:{color}"></i> {val:.2f}<br>'
        if i == 0: legend_html += '&nbsp;.<br>&nbsp;.<br>' # Kis térköz
    legend_html += '''
    </div>
    '''
    map_price_per_sqm_circles_red.get_root().html.add_child(folium.Element(legend_html))

    print("Ár/nm piros kör diagram elkészült (matplotlib szükséges).")
else:
    if 'matplotlib' not in sys.modules:
         print("Skipping CircleMarker map due to missing matplotlib.")
    else:
        print("Hiba: Nincs adat az Ár/nm kör diagramhoz.")
try:
    map_filename = "high_price_map.html"
    map_price_per_sqm_circles_red.save(map_filename)
    print(f"Térkép sikeresen mentve ide: {map_filename}")
except Exception as e:
    print(f"Hiba történt a térkép mentése közben: {e}")


--- Térkép: Ár/nm Színezett Körökkel (Piros Skála) ---


<ipython-input-24-d86b62a0cd6f>:22: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('Reds') # <- ITT VÁLTOZTATTUK A SZÍNSKÁLÁT


Ár/nm piros kör diagram elkészült (matplotlib szükséges).
Térkép sikeresen mentve ide: high_price_map.html


Térkép sikeresen mentve ide: high_price_map.html
